In [10]:
from gurobipy import Model, GRB

# ========= Données =========
# 6 universités, 5 critères: [salary_1y, salary_5y, %intl, %<3m, %PhD]
X = [
    [27.5, 30.0,  8.0,  83.0, 55.0],   # U1
    [32.5, 37.5, 45.0,  45.0, 91.5,],   # U2
    [25.0, 32.5, 16.0,  90.0, 25.0],   # U3
    [30.0, 35.0,  4.0,  75.0, 85.0],   # U4
    [25.0, 32.5, 24.0, 100.0,100.0],   # U5
    [39.0, 40.0,  8.0, 100.0, 15.0],   # U6
]
m, n = len(X), len(X[0])

# Échelles (bornes des critères)
mins = [min([X[j][i] for j in range(m)]) for i in range(n)]
maxs = [max([X[j][i] for j in range(m)]) for i in range(n)]

# L = 3 morceaux (=> 4 points de rupture) pour chaque critère
L = [30]*n

# Classement strict fourni: U1 ≻ U2 ≻ U3 ≻ U4 ≻ U5 ≻ U6
order = [0, 1, 2, 3, 4, 5]
EPS = 1e-4  # marge stricte

# ========= Utilitaires =========
def breakpoints(mi, Ma, Li):
    dx = (Ma - mi) / Li
    return [mi + k*dx for k in range(Li+1)]

def locate_segment_and_lambda(x, mi, Ma, Li):
    xs = breakpoints(mi, Ma, Li)
    if x <= xs[0]:   return 1, 0.0
    if x >= xs[-1]:  return Li, 1.0
    for k in range(1, Li+1):
        if xs[k-1] <= x <= xs[k]:
            denom = xs[k] - xs[k-1]
            lam = 0.0 if denom == 0 else (x - xs[k-1]) / denom
            return k, lam
    return Li, 1.0  # fallback

# Pré-calcul des (k, λ) pour tous (i,j)
seg_lambda = [[None]*m for _ in range(n)]
for i in range(n):
    for j in range(m):
        k, lam = locate_segment_and_lambda(X[j][i], mins[i], maxs[i], L[i])
        seg_lambda[i][j] = (k, lam)

# ========= Modèle UTA (LP) =========
model = Model("UTA_inference_L3")
model.Params.OutputFlag = 0

# u_{i}^{k} = s_i(x_i^k), k=0..L[i]
u = []
for i in range(n):
    ui = []
    for k in range(L[i]+1):
        ui.append(model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name=f"u_{i}_{k}"))
    u.append(ui)

# Normalisation
for i in range(n):
    model.addConstr(u[i][0] == 0.0, name=f"norm_min_{i}")
model.addConstr(sum(u[i][L[i]] for i in range(n)) == 1.0, name="sum_weights")

# Monotonicité
for i in range(n):
    for k in range(1, L[i]+1):
        model.addConstr(u[i][k] >= u[i][k-1], name=f"mono_{i}_{k}")

# Erreurs (L1)
sigma_p = [model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"sigma_p_{j}") for j in range(m)]
sigma_m = [model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"sigma_m_{j}") for j in range(m)]

# s'(j) = sum_i s_i(x_{i,j}) - sigma^+ + sigma^-
s_prime = []
for j in range(m):
    expr = 0.0
    for i in range(n):
        k, lam = seg_lambda[i][j]
        expr += (1 - lam) * u[i][k-1] + lam * u[i][k]
    expr = expr - sigma_p[j] + sigma_m[j]
    s_prime.append(expr)

# Contraintes de classement (consécutives)
for t in range(len(order)-1):
    j  = order[t]
    jn = order[t+1]
    model.addConstr(s_prime[j] >= s_prime[jn] + EPS, name=f"rank_{j}_gt_{jn}")

# Objectif: min sum_j (sigma^+ + sigma^-)
model.setObjective(sum(sigma_p) + sum(sigma_m), GRB.MINIMIZE)

# Optimisation
model.optimize()

# ========= Lecture résultats =========
def score_of(x_vec):
    total = 0.0
    for i in range(n):
        k, lam = locate_segment_and_lambda(x_vec[i], mins[i], maxs[i], L[i])
        total += (1 - lam) * u[i][k-1].X + lam * u[i][k].X
    return total

# Poids = u_i^{L_i}
weights = [u[i][L[i]].X for i in range(n)]
print("Weights (w_i):", [round(w,6) for w in weights], "; Sum =", round(sum(weights),6))

# Ruptures et valeurs s_i
for i in range(n):
    xs = breakpoints(mins[i], maxs[i], L[i])
    vals = [u[i][k].X for k in range(L[i]+1)]
    print(f"\nCriterion C{i+1}")
    print(" breakpoints:", [round(x,4) for x in xs])
    print(" s_i values :", [round(v,6) for v in vals])

# Scores & ranking
scores = [score_of(X[j]) for j in range(m)]
ranking = sorted(range(m), key=lambda j: scores[j], reverse=True)

print("\nScores:")
for j in range(m):
    print(f"U{j+1}: {scores[j]:.6f}")
print("Ranking (best→worst):", [f"U{j+1}" for j in ranking])

# Score de la nouvelle université U7 (32, 35, 25, 85, 12)
U7 = [32.0, 35.0, 25.0, 85.0, 12.0]
print("\nU7 score (optional):", round(score_of(U7), 6))


Weights (w_i): [0.4996, 0.0, 0.0002, 0.0003, 0.4999] ; Sum = 1.0

Criterion C1
 breakpoints: [25.0, 25.4667, 25.9333, 26.4, 26.8667, 27.3333, 27.8, 28.2667, 28.7333, 29.2, 29.6667, 30.1333, 30.6, 31.0667, 31.5333, 32.0, 32.4667, 32.9333, 33.4, 33.8667, 34.3333, 34.8, 35.2667, 35.7333, 36.2, 36.6667, 37.1333, 37.6, 38.0667, 38.5333, 39.0]
 s_i values : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.4996]

Criterion C2
 breakpoints: [30.0, 30.3333, 30.6667, 31.0, 31.3333, 31.6667, 32.0, 32.3333, 32.6667, 33.0, 33.3333, 33.6667, 34.0, 34.3333, 34.6667, 35.0, 35.3333, 35.6667, 36.0, 36.3333, 36.6667, 37.0, 37.3333, 37.6667, 38.0, 38.3333, 38.6667, 39.0, 39.3333, 39.6667, 40.0]
 s_i values : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [5]:
from gurobipy import Model, GRB

# ========= Données =========
# 10 voitures, 6 critères: [max_speed(+), cons_city(-), cons_road(-), horse_power(+), space(+), price(-)]
cars = [
    ("Peugeot 505 GR",    173, 11.4, 10.01, 10, 7.88, 49500),
    ("Opel Record 2000",  176, 12.3, 10.48, 11, 7.96, 46700),
    ("Citroen Visa SE",   142,  8.2,  7.30,  5, 5.65, 32100),
    ("VW Golf GLS",       148, 10.5,  9.61,  7, 6.15, 39150),
    ("Citroen CX Pallas", 178, 14.5, 11.05, 13, 8.06, 64700),
    ("Mercedes 230",      180, 13.6, 10.40, 13, 8.47, 75700),
    ("BMW 520",           182, 12.7, 12.26, 11, 7.81, 68593),
    ("Volvo 244 DL",      145, 14.3, 12.95, 11, 8.38, 55000),
    ("Peugeot 104 ZS",    161,  8.6,  8.42,  7, 5.11, 35200),
    ("Citroen Dyane",     117,  7.2,  6.75,  3, 5.81, 24800),
]

# X brut (m x n)
X_raw = [list(r[1:]) for r in cars]
m, n = len(X_raw), len(X_raw[0])

# *** Transformation coûts -> bénéfices (monotonie croissante) ***
# colonnes coût (0-based): cons_city=1, cons_road=2, price=5
cost_cols = {1, 2, 5}
# on construit X en bénéfices (max - x) pour les colonnes coût
X = [row[:] for row in X_raw]
for i_col in cost_cols:
    col_max = max(row[i_col] for row in X_raw)
    for j in range(m):
        X[j][i_col] = col_max - X[j][i_col]

# Échelles (bornes des critères) sur X (bénéfices)
mins = [min(X[j][i] for j in range(m)) for i in range(n)]
maxs = [max(X[j][i] for j in range(m)) for i in range(n)]

# L = 10 morceaux (=> 11 points de rupture) pour chaque critère
L = [10]*n

# Classement strict fourni par le tableau (1→10 déjà trié) :
order = list(range(m))  # [0,1,2,3,4,5,6,7,8,9]
EPS = 1e-4  # marge stricte

# ========= Utilitaires =========
def breakpoints(mi, Ma, Li):
    dx = (Ma - mi) / Li if Ma != mi else 0.0
    return [mi + k*dx for k in range(Li+1)]

def locate_segment_and_lambda(x, mi, Ma, Li):
    xs = breakpoints(mi, Ma, Li)
    if x <= xs[0]:   return 1, 0.0
    if x >= xs[-1]:  return Li, 1.0
    for k in range(1, Li+1):
        if xs[k-1] <= x <= xs[k]:
            denom = xs[k] - xs[k-1]
            lam = 0.0 if denom == 0 else (x - xs[k-1]) / denom
            # clip défensif
            lam = 0.0 if lam < 0 else (1.0 if lam > 1 else lam)
            return k, lam
    return Li, 1.0  # fallback

# Pré-calcul des (k, λ) pour tous (i,j) sur X (bénéfices)
seg_lambda = [[None]*m for _ in range(n)]
for i in range(n):
    for j in range(m):
        k, lam = locate_segment_and_lambda(X[j][i], mins[i], maxs[i], L[i])
        seg_lambda[i][j] = (k, lam)

# ========= Modèle UTA (LP) =========
model = Model("UTA_cars_L3_same_logic")
model.Params.OutputFlag = 0

# u_{i}^{k} = s_i(x_i^k), k=0..L[i]
u = []
for i in range(n):
    ui = []
    for k in range(L[i]+1):
        ui.append(model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name=f"u_{i}_{k}"))
    u.append(ui)

# Normalisation
for i in range(n):
    model.addConstr(u[i][0] == 0.0, name=f"norm_min_{i}")
model.addConstr(sum(u[i][L[i]] for i in range(n)) == 1.0, name="sum_weights")

# Monotonicité
for i in range(n):
    for k in range(1, L[i]+1):
        model.addConstr(u[i][k] >= u[i][k-1], name=f"mono_{i}_{k}")

# Erreurs (L1) par alternative (même logique que ton code universités)
sigma_p = [model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"sigma_p_{j}") for j in range(m)]
sigma_m = [model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name=f"sigma_m_{j}") for j in range(m)]

# s'(j) = sum_i s_i(x_{i,j}) - sigma^+ + sigma^-  (sur X bénéfices)
s_prime = []
for j in range(m):
    expr = 0.0
    for i in range(n):
        k, lam = seg_lambda[i][j]
        expr += (1 - lam) * u[i][k-1] + lam * u[i][k]
    expr = expr - sigma_p[j] + sigma_m[j]
    s_prime.append(expr)

# Contraintes de classement (consécutives)
for t in range(len(order)-1):
    j  = order[t]
    jn = order[t+1]
    model.addConstr(s_prime[j] >= s_prime[jn] + EPS, name=f"rank_{j}_gt_{jn}")

# Objectif: min sum_j (sigma^+ + sigma^-)
model.setObjective(sum(sigma_p) + sum(sigma_m), GRB.MINIMIZE)

# Optimisation
model.optimize()

# ========= Lecture résultats =========
def score_of(x_vec):
    """Prend un vecteur déjà converti en bénéfices (mêmes échelles que X/mins/maxs)."""
    total = 0.0
    for i in range(n):
        k, lam = locate_segment_and_lambda(x_vec[i], mins[i], maxs[i], L[i])
        total += (1 - lam) * u[i][k-1].X + lam * u[i][k].X
    return total

# helper: convertit un vecteur brut (coûts+­bénéfices) en bénéfices pour score_of
def to_benefit(x_raw):
    x = x_raw[:]
    # appliquer (max - x) sur colonnes coût, avec les mêmes max que dans X_raw
    # attention: on doit utiliser les mêmes max que ceux utilisés pour construire X
    col_max = [max(row[i] for row in X_raw) for i in range(n)]
    for i_col in cost_cols:
        x[i_col] = col_max[i_col] - x[i_col]
    return x

# Poids = u_i^{L_i}
weights = [u[i][L[i]].X for i in range(n)]
print("Weights (w_i):", [round(w,6) for w in weights], "; Sum =", round(sum(weights),6))

# Ruptures et valeurs s_i (sur l'échelle "bénéfices")
for i in range(n):
    xs = breakpoints(mins[i], maxs[i], L[i])
    vals = [u[i][k].X for k in range(L[i]+1)]
    print(f"\nCriterion C{i+1}")
    print(" breakpoints (benefits):", [round(x,6) for x in xs])
    print(" s_i values            :", [round(v,6) for v in vals])

# Scores & ranking des 10 voitures
scores = [score_of(to_benefit(X_raw[j])) for j in range(m)]
ranking = sorted(range(m), key=lambda j: scores[j], reverse=True)

print("\nScores:")
for j in range(m):
    print(f"{j+1:02d}. {cars[j][0]:20s}: {scores[j]:.6f}")
print("Ranking (best→worst):", [cars[j][0] for j in ranking])

# Score de la Fiat Panda (134, 7.4, 6.9, 3, 5.52, 25200)
panda_raw = [134, 7.4, 6.9, 3, 5.52, 25200]
panda_score = score_of(to_benefit(panda_raw))
all_scores = scores + [panda_score]
all_names  = [r[0] for r in cars] + ["Fiat Panda"]
order_with_panda = sorted(range(len(all_scores)), key=lambda j: all_scores[j], reverse=True)
panda_pos = order_with_panda.index(len(all_scores)-1) + 1

print(f"\nFiat Panda score: {panda_score:.6f}")
print(f"Fiat Panda position among the 10: {panda_pos}/11 (if inserted).")
print("Order with Panda:", " > ".join(all_names[j] for j in order_with_panda))


Weights (w_i): [0.00019, 0.498991, 0.0, 0.49974, 0.000541, 0.000538] ; Sum = 1.0

Criterion C1
 breakpoints (benefits): [117.0, 123.5, 130.0, 136.5, 143.0, 149.5, 156.0, 162.5, 169.0, 175.5, 182.0]
 s_i values            : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00019, 0.00019, 0.00019]

Criterion C2
 breakpoints (benefits): [0.0, 0.73, 1.46, 2.19, 2.92, 3.65, 4.38, 5.11, 5.84, 6.57, 7.3]
 s_i values            : [0.0, 0.0, 0.0, 0.0, 0.000101, 0.000101, 0.000101, 0.000101, 0.000101, 0.000597, 0.498991]

Criterion C3
 breakpoints (benefits): [0.0, 0.62, 1.24, 1.86, 2.48, 3.1, 3.72, 4.34, 4.96, 5.58, 6.2]
 s_i values            : [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Criterion C4
 breakpoints (benefits): [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0]
 s_i values            : [0.0, 0.0, 0.49949, 0.49949, 0.49949, 0.4996, 0.4996, 0.4996, 0.4996, 0.49974, 0.49974]

Criterion C5
 breakpoints (benefits): [5.11, 5.446, 5.782, 6.118, 6.454, 6.79, 7.126, 7.462,